# 作业

**Q1: 怎么在ip被封之后实现自动更换代理池内的代理？**

A1: 用random.choice 随机选取ip

**Q2: 如何用一句通俗的语言解释清楚request、beautifulsoup和selenium三者与浏览器之间的关系？**

A2: BeautifulSoup：处理速度快，同时可以**连续**查找，主要**用于静态网页** 

经过BeautifulSoup处理以后，编码方式都变成了Unicode,需要将其变成所需的编码方式：可以利用encode(‘需要的编码’)，还可以利用 BeautifulSoup(网页/html, lxml/xml”).prettify(‘需要的编码’) 可以利用soup.original_encoding检测原来的编码。
Selenium：主要用于**动态网页**，查找速度慢，解析时要注意 .find_**elements**_by_xpath和.find_**element**_by_xpath有区别，同时利用浏览器时要配置。 .PhantomJS： 
drive=webdriver.PhantomJS(‘D:\Anaconda2\phantomjswindow**s\bin**phantomjs.exe’) 

**Q3: 构建好代理池后，如何在一次爬虫中自动切换代理？ 比如代理无效，或者代理ip被封，这时自动切换下一个ip。**

A3: 首先你要有一个ip代理池（如果比较豪可以自己买真实ip自行搭建，好处独享有技术门槛,或者找第三方ip代理商对接,好吃廉价,但ip不独享）， 真实ip需要自己写程序来代理转发，第三方ip代理商则会提供相关转发API,直接调用就可以，这东西没什么技术难度 

**Q4: ip_list.append(f'{protpcol}://{ip}:{port}')这里的f是格式化？**

A4:

1. 从代理*ip*网站爬取*IP*地址及端口号并储存
2. 验证*ip*是否能用
3. 格式化*ip*地址
4. 在*requests*中使用代理*ip*爬取网站

# 小项目
1. 挑战项目：模拟登录丁香园，并抓取论坛页面所有的人员基本信息与回复帖子内容。
2. 丁香园论坛：[http://www.dxy.cn/bbs/thread/626626#626626](http://www.dxy.cn/bbs/thread/626626#626626) 。

In [1]:
import requests, json, re, random,time
from bs4 import BeautifulSoup
from selenium import webdriver
from lxml import etree

In [2]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By

In [86]:

"""
使用selenium进行模拟登陆
1.初始化ChromDriver
2.丁香园论坛页面
3.找到用户名的输入框，输入用户名
4.找到密码框，输入密码
5.提交用户信息
"""
name = '*'
passwd = '*'
driver = webdriver.Chrome('./chromedriver')
driver.get('http://www.dxy.cn/bbs/thread/626626#626626')
# 将窗口调整最大
driver.maximize_window()
# 休息5s
time.sleep(5)
current_window_1 = driver.current_window_handle
print(current_window_1)

CDwindow-C14469269D37178F79622876C99ED52A


In [88]:
button = driver.find_element_by_xpath("/html/body/div/div[3]/div[4]/div/div/a[2]")
button.click()

In [90]:
# phone = driver.find_element_by_name('username')
phone = driver.find_element_by_xpath('//input[@name="username"]')
phone.send_keys(name)
# password = driver.find_element_by_name('password')
password = driver.find_element_by_xpath("//input[@name='password']")
password.send_keys(passwd)
submit = driver.find_element_by_xpath("/html/body/div[2]/div[2]/div[3]/form/div[1]/div[3]/button")
# submit = driver.find_element_by_id("dologin")
time.sleep(15)
submit.click()
time.sleep(10)
# print(driver.page_source)

In [91]:
html = driver.page_source

#### 2. lxml解析html

In [92]:
tree = etree.HTML(html) 
tree

<Element html at 0x1eedb856788>

#### 3. 利用Xpath表达式获取user和content（完成xpath的语句）


In [93]:
user = tree.xpath('//div[@class="auth"]/a/text()')[:-1]
user = [x.strip() for x in user]
# print(user)
content = tree.xpath('//td[@class="postbody"]')
content = [x.xpath('string(.)').strip() for x in content]

In [94]:
level = tree.xpath('//div[@class="user-level-area"]/text()')[:-1]

In [95]:
num = tree.xpath('//div[@class="num"]/a/text()')[:-3]
num = [num[i:i+3] for i in range(0, len(num), 3)]

In [96]:
len(user), len(content), len(level), len(num)

(27, 27, 25, 27)

#### 4. 保存爬取的内容

In [99]:
results = list(zip(range(1, len(user) + 1, 1), user, level, num, content))[:3]

[(1,
  '楼医生',
  '常驻站友',
  ['3', '2', '182'],
  '我遇到一个“怪”病人，向大家请教。她，42岁。反复惊吓后晕厥30余年。每次受响声惊吓后发生跌倒，短暂意识丧失。无逆行性遗忘，无抽搐，无口吐白沫，无大小便失禁。多次跌倒致外伤。婴儿时有惊厥史。入院查体无殊。ECG、24小时动态心电图无殊；头颅MRI示小软化灶；脑电图无殊。入院后有数次类似发作。请问该患者该做何诊断，还需做什么检查，治疗方案怎样？'),
 (2,
  'lion000',
  '丁香园准中级站友',
  ['43', '9', '82'],
  '从发作的症状上比较符合血管迷走神经性晕厥，直立倾斜试验能协助诊断。在行直立倾斜实验前应该做常规的体格检查、ECG、UCG、holter和X-ray胸片除外器质性心脏病。贴一篇“口服氨酰心安和依那普利治疗血管迷走性晕厥的疗效观察”作者：林文华 任自文 丁燕生http://www.ccheart.com.cn/ccheart_site/Templates/jieru/200011/1-1.htm'),
 (3,
  'xghrh',
  '常驻站友',
  ['68', '19', '412'],
  '同意lion000版主的观点：如果此患者随着年龄的增长，其发作频率逐渐减少且更加支持，不知此患者有无这一特点。入院后的HOLTER及血压监测对此患者只能是一种安慰性的检查，因在这些检查过程中患者发病的机会不是太大，当然不排除正好发作的情况。对此患者应常规作直立倾斜试验，如果没有诱发出，再考虑有无可能是其他原因所致的意识障碍，如室性心动过速等，但这需要电生理尤其是心腔内电生理的检查，毕竟是有一种创伤性方法。因在外地，下面一篇文章可能对您有助，请您自己查找一下。心理应激事件诱发血管迷走性晕厥1例 ，杨峻青、吴沃栋、张瑞云，中国神经精神疾病杂志， 2002 Vol.28 No.2')]

In [101]:
# 打印爬取的结果
print('楼层\t\t名字\t\t级别\t\t积分, 得票, 叮当\t\t评论')

for i in results:
    print(f'post-{i[0] + 1}\t\t{i[1]}\t\t{i[2]}\t\t{i[3]}\t\t\n{i[4]}')
    print('')
    print("*"*100)
    print('')

楼层		名字		级别		积分, 得票, 叮当		评论
post-1		楼医生		常驻站友		['3', '2', '182']		
我遇到一个“怪”病人，向大家请教。她，42岁。反复惊吓后晕厥30余年。每次受响声惊吓后发生跌倒，短暂意识丧失。无逆行性遗忘，无抽搐，无口吐白沫，无大小便失禁。多次跌倒致外伤。婴儿时有惊厥史。入院查体无殊。ECG、24小时动态心电图无殊；头颅MRI示小软化灶；脑电图无殊。入院后有数次类似发作。请问该患者该做何诊断，还需做什么检查，治疗方案怎样？

****************************************************************************************************

post-2		lion000		丁香园准中级站友		['43', '9', '82']		
从发作的症状上比较符合血管迷走神经性晕厥，直立倾斜试验能协助诊断。在行直立倾斜实验前应该做常规的体格检查、ECG、UCG、holter和X-ray胸片除外器质性心脏病。贴一篇“口服氨酰心安和依那普利治疗血管迷走性晕厥的疗效观察”作者：林文华 任自文 丁燕生http://www.ccheart.com.cn/ccheart_site/Templates/jieru/200011/1-1.htm

****************************************************************************************************

post-3		xghrh		常驻站友		['68', '19', '412']		
同意lion000版主的观点：如果此患者随着年龄的增长，其发作频率逐渐减少且更加支持，不知此患者有无这一特点。入院后的HOLTER及血压监测对此患者只能是一种安慰性的检查，因在这些检查过程中患者发病的机会不是太大，当然不排除正好发作的情况。对此患者应常规作直立倾斜试验，如果没有诱发出，再考虑有无可能是其他原因所致的意识障碍，如室性心动过速等，但这需要电生理尤其是心腔内电生理的检查，毕竟是有一种创伤性方法。因在外地，下面一篇文章可能对您有助，请您自己查找一下。心理应激事件诱发血管迷走性晕厥1例 ，杨峻青、吴沃栋、张瑞云，中国神经精

In [104]:
driver.quit()